In [1]:
# -*- coding: utf-8 -*-
import sys; print('Python %s on %s' % (sys.version, sys.platform))
import os
import time
import json
import re
import pandas as pd
import numpy as np
from glob import glob, iglob
from tqdm import tqdm
from datetime import datetime
import matplotlib.pyplot as plt

Python 3.8.10 (default, May 26 2023, 14:05:08) 
[GCC 9.4.0] on linux


In [2]:
import utils
import gpt
from sklearn.metrics.pairwise import cosine_similarity

DATASET_FOLDER = './dataset/'
OUTPUT_FOLDER = './output/'

LIBRARY = 'gs'
CATEGORY = 'jppy'

## meta

In [3]:
df_meta = pd.read_parquet(OUTPUT_FOLDER + f'meta-{LIBRARY}-{CATEGORY}.parquet')
print(df_meta.shape)

(73, 9)


In [4]:
df_meta.head(2)

,library,category,file_name,file_path,external_link,chapter,text_length,token_count,content
idx,,,,,,,,,
87c5c645171300baa4b6df99bb5ef28fa04fa265b666349e2c1375b7962a44ef,gs,jppy,Payroll Basics,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,1,1467,271,# Payroll Basics\nPurpose\nThe Payroll is base...
ebd0e02c76753f2896df4795a0155a6e0afab2d339a73924dfa7ee03b3a759bf,gs,jppy,Payroll Driver,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,2,993,210,# Payroll Driver\nDefinition\nProgram you use ...


In [5]:
df_meta[['content']]

,content
idx,
87c5c645171300baa4b6df99bb5ef28fa04fa265b666349e2c1375b7962a44ef,# Payroll Basics\nPurpose\nThe Payroll is base...
ebd0e02c76753f2896df4795a0155a6e0afab2d339a73924dfa7ee03b3a759bf,# Payroll Driver\nDefinition\nProgram you use ...
94deab0475a95f52e10a142b892d67798df01e73fa1d354e4827fd3f5cbd37e2,# Payroll Schema\nDefinition\nThe payroll sche...
30d900bbb4076d7df560b82c33b38112afe91b805ae192cec68c5a29cc1bd4c3,# Payroll-Relevant Files\nDefinition\nPayroll ...
10df96aa580c23d5bf3f03a09e30f81b6efdbe375b7b8973f58caff91f31d545,# Internal Tables for Payroll\nDefinition\nTab...
...,...
d054628a73749896c69db8f72eb6a4b455740518a996610099d3545e12826db2,# Administrating Employee's Nursing Insurance ...
239f3d7a733eefe3b0da8ed926ce3f4799a747f1876d2ad5419e9156e8c1a8bd,# Maintaining Nursing Insurance Status per Per...
6fbd881bca11bbfbd7420aad085dee4666c928d710ebd0a403ec957b6ffe5dcb,# Maintaining Nursing Insurance Status by Peri...


In [6]:
df_meta[df_meta['token_count']<10]

,library,category,file_name,file_path,external_link,chapter,text_length,token_count,content
idx,,,,,,,,,
fe32709c55435dc70032d4c0c72d57231393161292ebefcb2be217824af2475c,gs,jppy,Infotypes for Payroll Japan,./dataset/gs/JPPY/jppyp1.md,https://help.sap.com/docs/SAP_S4HANA_ON-PREMIS...,22,30,8,# Infotypes for Payroll Japan\n


## embedding validation

In [7]:
df_embedding = pd.read_parquet(OUTPUT_FOLDER + f'embedding-{LIBRARY}-{CATEGORY}.parquet')
print(df_embedding.shape)

(73, 1536)


In [8]:
embedder = gpt.Embedding()
for idx, item in df_embedding.sample(10).iterrows():    
    plain_text = df_meta.loc[idx].content
#     print(plain_text)
    print(idx, len(list(item)), len(plain_text))
    embedding = embedder.run(plain_text).result_
    
    similarity = cosine_similarity(np.array(item).reshape(1, -1), np.array(embedding).reshape(1, -1))
    assert similarity[0][0] - 1 < 0.00001
    time.sleep(1)

f82fba9bb6eb3aefe367cd5377615b9258fb86085c1f0b5c1c8c6983e23f8619 1536 1473
e4e989b2168c0d9e63610c0fad5777659f612b695234da8a32ddfbe75f7dc226 1536 2966
30d900bbb4076d7df560b82c33b38112afe91b805ae192cec68c5a29cc1bd4c3 1536 933
fe32709c55435dc70032d4c0c72d57231393161292ebefcb2be217824af2475c 1536 30
c3c1d44e345ca4c0b795a74adadac7f668631afacab0d50c85eec06c39cf2e5a 1536 3267
a3e24ecfd031f38e5713fe47570cb50e1d40753ac95c4336d3d2e4a2176fb03e 1536 1360
2af820c0e3a77fd9e0816c8e9f013f3c2927e667cd5fb784051467e8e4cbce32 1536 982
7918a37865bcf619456ec368fe915becf10ed409fcfd5ef74d395088f1b171ce 1536 1094
17f947b4de763bdde0cb4df9e5604c3bb68460f8153072638050de74a9ef8c39 1536 1416
b46685aced2bde13d36a3e1d739ce48c071c8046cd2131c2ee4cd667b2ebc96a 1536 1229
